### Usando Tweepy

Para la ejecución de este código necesitarás haber instalado la librería
`tweepy` ([docu](https://docs.tweepy.org/en/stable/client.html)). La ventaja
de usar `tweepy` es el ahorro en el esfuerzo de construir clases para representar
la estructura de datos de Twitter, además de beneficiarnos de los wrappers de
los métodos de adquisición de datos expuestos por Twitter (algunos de los cuales
ya hemos visto en el [tutorial anterior](twitter-api-v2-raw.ipynb)).

Al igual que antes, requeriremos el tener los credenciales en un fichero
`JSON`, de donde los leeremos para la creación de los objetos OAuth:

In [6]:
import tweepy
import json

def get_credentials(file: dict):
    """Reads credential from json file
    :param file: JSON file with CONSUMER and ACCESS TOKE and SECRET values
    :return: dict
    """
    with open(file, "r") as file:
        result = json.load(file)
        return result

creds = get_credentials(file="../../auth/twitter_credentials.json")

api_v2 = tweepy.Client(
    bearer_token=creds["BEARER_TOKEN"],
    consumer_key=creds["CONSUMER_KEY"],
    consumer_secret=creds["CONSUMER_SECRET"],
    access_token=creds["ACCESS_TOKEN"],
    access_token_secret=creds["ACCESS_SECRET"],
    wait_on_rate_limit=True,
)

Lo primero que podemos hacer es una búsqueda, para así comprobar la sencillez
que ofrece:

In [7]:
query = 'from:suhemparack -is:retweet'
tweet_fields = [
    'author_id',
    'id',
    'context_annotations',
    'created_at',
]
max_results = 10
tweets = api_v2.search_recent_tweets(
    query=query,
    tweet_fields=tweet_fields,
    max_results=max_results,
)
tweets.data

[<Tweet id=1488573858198130690 text=Do you use Twarc from @documentnow for getting data from #TwitterAPI for your research? Join us this Friday to learn from Twitter Developer Insider @IgorBrigadir who is a contributor (and super user) of Twarc to learn how you can get the most out of it 📊 https://t.co/TupdnLCXCB>,
 <Tweet id=1488559232324562945 text=@Cataranea @noamross BTW academic researchers can get data from the full-archive of public Tweets (i.e. older than 7 days) for free Check out 👉https://t.co/xRPnqYBUkV>,
 <Tweet id=1487958483060199433 text=Happy (belated) 1st Birthday to the academic research product track of the #TwitterAPI v2 🎂🎉🥳
 
 Over the last year, researchers have used it to study the conversation around covid, misinformation, polarization and more
 
 How are you using it for your research? https://t.co/uZLQtE3zM1>,
 <Tweet id=1487537194671558657 text=10K downloads for the academictwitteR package 🔥🔥 Well done @justin_ct_ho @cbarrie @chainsawriot 👏👏 https://t.co/O1a1G

Así podemos seguir con la búsqueda de followers:

In [8]:
user_id = "398306220"
max_results = 10
followers = api_v2.get_users_followers(
    id=user_id,
)
followers.data

Rate limit exceeded. Sleeping for 365 seconds.


[<User id=1488641872302067720 name=n0s3 j4j4 username=lulitauwuxdd>,
 <User id=1488642278763769857 name=cuentasreynaldoarturo@gmail.com username=exdiuwu>,
 <User id=1488639859363299334 name=Facundo username=Facundo72605527>,
 <User id=156496195 name=freddy tempos username=freddy_t1974>,
 <User id=1488641281563807744 name=Jesus Cardenas username=JesusCa65721388>,
 <User id=1252022284023447552 name=Mariana Ramírez username=Mariana70307709>,
 <User id=1488641554814275585 name=Mario Ruiz Alonso username=marioruizalonso>,
 <User id=1433864785825878018 name=ASHURA光 username=FelipeF96322147>,
 <User id=1487908674924421122 name=Vamy_jamón username=VamyJamon>,
 <User id=1446271986380988423 name=Julieta ‽ username=julisusaye>,
 <User id=1488641324119207947 name=Leonel Gonzalez username=Leo305xd>,
 <User id=1308585128512442368 name=nachoo username=naxoossj>,
 <User id=1488640819741564938 name=Alan Pineda username=AlanPienda01>,
 <User id=1488641630919958533 name=Pedro Antonio Bas Barrera username

El número máximo por página de resultados (ver [paginación](https://developer.twitter.com/en/docs/twitter-api/pagination))
es 100. Esto significa que si queremos acceder a más de 100 followers, tendremos que de alguna
forma acceder a la siguiente página. Para ello, la API de Tweeter nos devuelve en sus resultados
una parte de `metadata`, donde nos da un token que debemos pasar en la próxima petición.
Trabajando con tweepy, esto se hace de la siguiente forma:

In [ ]:
max_items = 10
max_results = 5
tweets = []
for tweet in tweepy.Paginator(
        api_v2.get_users_followers,
        id=user_id,
        max_results=max_results).flatten(limit=max_items):
    tweets += [tweet]

len(tweets)

También podemos buscar tweets recientes, como ya hicimos con la API directamente:


In [8]:
query: str = "djokovic -is:retweet lang:en"
tweet_fields = [
    'context_annotations',
    'created_at',
    'author_id',
]
max_results = 10
max_items = 50

tweets = [
    tweet
    for tweet in tweepy.Paginator(
        api_v2.search_recent_tweets,
        query=query,
        tweet_fields=tweet_fields,
        max_results=max_results).flatten(limit=max_items)
]
tweets

In [ ]:
# Replace with your own search query
import pandas as pd
import plotly.express as px

data = [{"date": d.created_at, "text": d.text} for d in tweets]
words = [d["text"].split() for d in data]
words = sum(words, [])

df = pd.DataFrame({"words": words})
fig = px.histogram(df, x="words").update_xaxes(categoryorder="total descending")
fig.show()

Otro concepto bastante útil para analítica son los *timelines*, que básicamente
nos permiten acceder a los tweets de un usuario en un intervalo de tiempo:

In [ ]:
import datetime

def get_timeline(
        id: str,
        tweet_fields: list,
        max_results: int,
        max_items: int,
        start_time: datetime,
):
    tweets = [
        tweet
        for tweet in tweepy.Paginator(
            api_v2.get_users_tweets,
            id=id,
            start_time=start_time.strftime("%Y-%m-%dT00:00:00+00:00"),
            tweet_fields=tweet_fields,
            max_results=max_results).flatten(limit=max_items)
    ]

    return tweets

timeline = get_timeline(
    id="398306220",
    tweet_fields=[
        "id",
        "created_at",
        "public_metrics",
        "context_annotations",
    ],
    max_results=10,
    max_items=20,
    start_time=(datetime.datetime.now(datetime.timezone.utc)
                - datetime.timedelta(days=7))
)

timeline